In [ ]:
import sys
sys.path.insert(0, "..")

from src.talent_selection_flow.main import TalentSelectionFlow

In [ ]:
flow = TalentSelectionFlow()

In [ ]:
# await kickoff_async for notebooks
await flow.kickoff_async(inputs={"raw_input": "tot el camp és un clam, som la gent blaugrana"})